I have used Google Colaboratory and the dataset on which i performed ingestion and validation is Old-newspaper.tsv from kaggle

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
!ls

drive  sample_data


In [2]:
import timeit # to find out the execution time

In [4]:
setup='import pandas as pd'
stmtcode='''
path="/content/drive/MyDrive/Colab Notebooks/data/old-newspaper.tsv"
df=pd.read_csv(path, sep='\t')
'''
print('It took', timeit.timeit(stmtcode, setup=setup, number=1), 'seconds for pandas to read the file')

It took 144.470565299 seconds for pandas to read the file


In [5]:
pip install "dask[dataframe]"

     |████████████████████████████████| 112kB 6.0MB/s 


In [6]:
setup='import dask.dataframe as dd'
stmtcode='''
path="/content/drive/MyDrive/Colab Notebooks/data/old-newspaper.tsv"
daskdf=dd.read_csv(path, sep='\t')
'''
print('It took', timeit.timeit(stmtcode, setup=setup, number=1), 'seconds for dask to read the file')

It took 0.2246963519999099 seconds for dask to read the file


In [ ]:
pip install modin

     |████████████████████████████████| 583kB 5.7MB/s 
     |████████████████████████████████| 9.9MB 8.9MB/s 
ERROR: google-colab 1.0.0 has requirement pandas~=1.1.0; python_version >= "3.0", but you'll have pandas 1.2.3 which is incompatible.
  Found existing installation: pandas 1.1.5
    Uninstalling pandas-1.1.5:
      Successfully uninstalled pandas-1.1.5


In [ ]:
!pip install modin[ray]

     |████████████████████████████████| 583kB 25.8MB/s 
     |████████████████████████████████| 9.9MB 48.4MB/s 
     |████████████████████████████████| 17.2MB 227kB/s 
     |████████████████████████████████| 48.5MB 86kB/s 
     |████████████████████████████████| 204kB 54.8MB/s 
     |████████████████████████████████| 81kB 7.4MB/s 
     |████████████████████████████████| 1.3MB 36.3MB/s 
     |████████████████████████████████| 133kB 47.2MB/s 
     |████████████████████████████████| 81kB 9.1MB/s 
     |████████████████████████████████| 3.1MB 31.0MB/s 
     |████████████████████████████████| 71kB 7.8MB/s 
     |████████████████████████████████| 143kB 46.5MB/s 
     |████████████████████████████████| 296kB 45.4MB/s 
     |████████████████████████████████| 92kB 10.4MB/s 
  Created wheel for gpustat: filename=gpustat-0.6.0-cp37-none-any.whl size=12621 sha256=d14430726074e6c6f41fb6fe378ebdcb7ba6fcd111ae205f14359ca7137fbe79
  Stored in directory: /root/.cache/pip/wheels/48/b4/d5/fb5b7f1d040f2ff

In [ ]:
!pip install -U ray

     |████████████████████████████████| 49.7MB 88kB/s 
     |████████████████████████████████| 1.0MB 41.3MB/s 
  Found existing installation: protobuf 3.12.4
    Uninstalling protobuf-3.12.4:
      Successfully uninstalled protobuf-3.12.4
  Found existing installation: ray 1.1.0
    Uninstalling ray-1.1.0:
      Successfully uninstalled ray-1.1.0


In [ ]:
previous = '''
import pandas as pd
import ray
ray.shutdown()
ray.init()
'''
code = '''
path="/content/drive/MyDrive/Colab Notebooks/data/old-newspaper.tsv"
data = pd.read_csv(path, sep='\t')
'''
print('Running Ray takes', timeit.timeit(code, setup=previous, number=1), 'seconds to read the file')

2021-05-03 19:43:44,121	INFO services.py:1269 -- View the Ray dashboard at http://127.0.0.1:8265


Running Ray takes 168.88544779900008 seconds to read the file


# In term of computational  efficiency:
1. Dask 0.22 seconds<br>
2. Modin with Ray 168.88 seconds <br>
3. Pandas 144 seconds <br>
Conclusion: Dask is most efficient

Schema **Validation**

---



In [7]:
import dask.dataframe as dd

In [8]:
path="/content/drive/MyDrive/Colab Notebooks/data/old-newspaper.tsv"
ddf = dd.read_csv(path, sep='\t')

In [ ]:
ddf.head()

,Language,Source,Date,Text
0,Afrikaans,republikein.com.na,2011/09/14,Die veranderinge aan die Britsgeboude Avensis ...
1,Afrikaans,republikein.com.na,2011/01/20,Duitsland se mans- en vrouespanne is die afgel...
2,Afrikaans,sake24.com,2009/11/28,"Mnr. Estienne de Klerk, uitvoerende direkteur ..."
3,Afrikaans,sake24.com,2009/11/12,Mustek is se finansiële-resultate-advertensie ...
4,Afrikaans,sake24.com,2011/02/04,nadat LMS se raad van trustees in Junie verled...


In [ ]:
ddf.columns

Index(['Language', 'Source', 'Date', 'Text'], dtype='object')

In [ ]:
ddf.columns.str.lower()

Index(['language', 'source', 'date', 'text'], dtype='object')

In [ ]:
ddf.isnull().sum().compute()

Language    0
Source      0
Date        0
Text        0
dtype: int64

Writing a utility a file

In [19]:
%%writefile utility.py
import logging
import os
import subprocess
import yaml
import dask.dataframe as dd
import datetime 
import gc
import re


################
# File Reading #
################

def read_config_file(filepath):
    with open(filepath, 'r') as stream:
        try:
            return yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            logging.error(exc)


def replacer(string, char):
    pattern = char + '{2,}'
    string = re.sub(pattern, char, string) 
    return string

def col_header_val(df,table_config):
    '''
    replace whitespaces in the column
    and standardized column names
    '''
    df.columns = df.columns.str.lower()
    expected_col = list(map(lambda x: x.lower(),  table_config['columns']))
    expected_col.sort()
    df.columns =list(map(lambda x: x.lower(), list(df.columns)))
    df = df.reindex(sorted(df.columns), axis=1)
    if len(df.columns) == len(expected_col) and list(expected_col)  == list(df.columns):
        print("column name and column length validation passed")
        return 1
    else:
        print("column name and column length validation failed")
        mismatched_columns_file = list(set(df.columns).difference(expected_col))
        print("Following File columns are not in the YAML file",mismatched_columns_file)
        missing_YAML_file = list(set(expected_col).difference(df.columns))
        print("Following YAML columns are not in the file uploaded",missing_YAML_file)
        logging.info(f'df columns: {df.columns}')
        logging.info(f'expected columns: {expected_col}')
        return 0

Overwriting utility.py


***Write YAML file***

In [20]:

%%writefile file.yaml
file_type: tsv
file_name: old-newspaper
inbound_delimiter: "\t"
outbound_delimiter: "|"
skip_leading_rows: 1
columns: 
    - Language
    - Source
    - Date
    - Text


Overwriting file.yaml


In [2]:
# Read config file
import utility as util
config_data = util.read_config_file("file.yaml")

In [3]:
config_data['file_name']

'old-newspaper'

In [3]:
#inspecting data of config file
config_data

{'columns': ['Language', 'Source', 'Date', 'Text'],
 'file_name': 'old-newspaper',
 'file_type': 'tsv',
 'inbound_delimiter': '\t',
 'outbound_delimiter': '|',
 'skip_leading_rows': 1}

In [7]:
import dask.dataframe as dd
path="/content/drive/MyDrive/Colab Notebooks/data/old-newspaper.tsv"
ddf = dd.read_csv(path, sep='\t')
ddf.head()

,Language,Source,Date,Text
0,Afrikaans,republikein.com.na,2011/09/14,Die veranderinge aan die Britsgeboude Avensis ...
1,Afrikaans,republikein.com.na,2011/01/20,Duitsland se mans- en vrouespanne is die afgel...
2,Afrikaans,sake24.com,2009/11/28,"Mnr. Estienne de Klerk, uitvoerende direkteur ..."
3,Afrikaans,sake24.com,2009/11/12,Mustek is se finansiële-resultate-advertensie ...
4,Afrikaans,sake24.com,2011/02/04,nadat LMS se raad van trustees in Junie verled...


In [4]:
# read the file using config file
import pandas as pd
file_type = config_data['file_type']
source_file = "/content/drive/MyDrive/Colab Notebooks/data/" + config_data['file_name'] + f'.{file_type}'
df = pd.read_csv(source_file,config_data['inbound_delimiter'])
df.head()


,Language,Source,Date,Text
0,Afrikaans,republikein.com.na,2011/09/14,Die veranderinge aan die Britsgeboude Avensis ...
1,Afrikaans,republikein.com.na,2011/01/20,Duitsland se mans- en vrouespanne is die afgel...
2,Afrikaans,sake24.com,2009/11/28,"Mnr. Estienne de Klerk, uitvoerende direkteur ..."
3,Afrikaans,sake24.com,2009/11/12,Mustek is se finansiële-resultate-advertensie ...
4,Afrikaans,sake24.com,2011/02/04,nadat LMS se raad van trustees in Junie verled...


In [5]:
#validate the header of the file
util.col_header_val(df,config_data)

column name and column length validation passed


1

In [6]:
print("columns of files are:" ,df.columns)
print("columns of YAML are:" ,config_data['columns'])

columns of files are: Index(['language', 'source', 'date', 'text'], dtype='object')
columns of YAML are: ['Language', 'Source', 'Date', 'Text']


**Pipeline to perform file Validation**

In [12]:
import dask.dataframe as dd

In [26]:
def DataPipeline(file_name: str) -> str:
  # 1. read the file using the YAML configuration file
  file_format = config_data['file_type']
  data = dd.read_csv(f'{file_name}.{file_type}', sep = config_data['inbound_delimiter'])

  # 2. validate columns
  if len(config_data['columns']) == len(data.columns) and list(config_data['columns']) == list(data.columns):
   
    return print('The file has passed the validation and can be compressed')
  else:
    print('The validation failed')

In [27]:
DataPipeline('/content/drive/MyDrive/Colab Notebooks/data/old-newspaper')

The file has passed the validation and can be compressed


In [30]:
def summary(file_name):

  file_type = config_data['file_type']
  data = dd.read_csv(f'{file_name}.{file_type}', sep = config_data['inbound_delimiter'])
  
  num_of_cols = len(data.columns)
  num_of_rows = len(data.iloc[:,0])
  import os
  file_size = os.path.getsize(f'{file_name}.{file_type}')

  return print(f'The file entered was {file_name}. It has {num_of_cols} columns and {num_of_rows} entries. It weighs {file_size / 1e9} GB.')

**Summary of the file**

In [31]:
summary('/content/drive/MyDrive/Colab Notebooks/data/old-newspaper')

The file entered was /content/drive/MyDrive/Colab Notebooks/data/old-newspaper. It has 4 columns and 16806041 entries. It weighs 6.024697599 GB.


In [35]:
def compression(file_name, new_file_name, format):

  import csv

  file_type = config_data['file_type']
  with open(f'{file_name}.{file_type}', 'r') as input_f:
    csv_reader = csv.reader(input_f, delimiter = config_data['inbound_delimiter'])

    with open(f'{new_file_name}.{format}', 'w') as output_f:
      csv_writer = csv.writer(output_f, delimiter = '|')

      for line in csv_reader:
        csv_writer.writerow(line)
      
      import gzip

      with open(f'{new_file_name}.{format}', 'rb') as input_cf:
        gzip.GzipFile(f'{new_file_name}.gz', 'wb')

In [36]:
compression('/content/drive/MyDrive/Colab Notebooks/data/old-newspaper', 'new_file', 'csv')

In [37]:
!pwd

/content


In [38]:
!ls

drive	   new_file.csv  __pycache__  utility.py
file.yaml  new_file.gz	 sample_data
